In [133]:

#<<<<<<<<<<<<<<<<<<<<#          Simulacion RS        #>>>>>>>>>>>>>>>>>>>># V 1.0


import random, numpy as np

TamañoDeLaMuestra = 500 # Cantidad de sujetos que quiero en la muestra.

In [134]:

# Defino a los agentes.

class Person (object):
    def __init__ (self, opinion,ID):
        self.Oi = opinion
        self.ID = ID # Genero un ID para cada agente
        self.IdeasVistas = [] # Me va a servir para almacenar las ideas a las que haya estado expuesto el sujeto
        self.IdeasVotadas = [] # Almaceno las ideas que voto

    
    def __str__ (self):
        try:
            devolver = 'despues defino que quiero que me devuelva'
            return devolver
        except:
            devolver = 'despues defino que quiero que me devuelva'
            return devolver
    
    def set_Oi (self, opinion):
        self.Oi = opinion
    
    def append_IdeasVistas (self, IdeaVista):
        self.IdeasVistas.append (IdeaVista)

    def append_IdeasVotadas (self, IdeaVotada):
        self.IdeasVotadas.append (IdeaVotada)
    
    def votar (self, ideas): # Esto esta mal, tengo que ver como hacerlo bien.
        Valores = []
        for i in ideas:
            V = abs((i-self.get_Oi)/self.get_Oi)
            Valores.append (V)
        Valores.sort()
        return ((Valores[0]*self.get_Oi)+self.get_Oi)
            
    def get_Oi (self):
        return self.Oi
    
    def get_IdeasVistas (self):
        return self.IdeasVistas

    def get_ID (self):
        return self.ID
    
    def clean_IdeasVistas (self):
        self.IdeasVistas = []
    
    


In [135]:

# Creo la muestra
Muestra_Completa = {}
for i in range (TamañoDeLaMuestra):
    Muestra_Completa [i] = Person (np.random.uniform (-1,1),i*111)


In [136]:

#¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤ Simulacion Basica ¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤#

ParticipanObligados = 10 # Estos van a ser los primeros sujetos, que opinan si o si.
QOpinionesExpuestas = 5 #Cantidad de opiniones que quiero que vea cada agente
OpinanNoVotan = 0
Muestra_Simulacion = Muestra_Completa.copy()
DataSave = np.zeros ([TamañoDeLaMuestra,4], dtype='int16') # Creo un array para almacenar los datos con: ID; Oi; Votos; Veces Expuesta.

for i in Muestra_Simulacion:
    if OpinanNoVotan < ParticipanObligados:
        DataSave [i,0] = Muestra_Simulacion.get(i).get_ID()
        DataSave [i,1] = Muestra_Simulacion.get(i).get_Oi()
    else:
        # ¿DataSave.sort(axis = )? Hacer que se ordene el array de menor a mayor segun la columna que almacena las vistas.
        Opiniones = DataSave [:QOpinionesExpuestas,1]
        #for i in Opiniones:
            #Hacer que se sumen la vista y el voto
        Muestra_Simulacion.get(i).votar(Opiniones) # Esto me devuelve la idea que fue votada, tengo que ver como volcarlo al array
        DataSave [i,0] = Muestra_Simulacion.get(i).get_ID()
        DataSave [i,1] = Muestra_Simulacion.get(i).get_Oi()

# Como accedo a una fila del array especifica acorde a la idea? osea en la fila que este la idea x, en la columna de al lado +=1
# A cada ID de sujeto le sumo la idea que voto tambien?